# Setup working directory


In [0]:
import os

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
WORKING_DIR=os.path.join('/content', 'drive', 'My Drive', 'Colab Notebooks', 'NLU')
os.path.exists(WORKING_DIR)

True

In [4]:
cd $WORKING_DIR

/content/drive/My Drive/Colab Notebooks/NLU


# Clone transformers repo and checkout version 2.7.0

In [5]:
!rm -rf transformers
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 26405 (delta 3), reused 3 (delta 0), pack-reused 26390
Receiving objects: 100% (26405/26405), 15.88 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (18396/18396), done.
Checking out files: 100% (675/675), done.


In [6]:
cd transformers/

/content/drive/My Drive/Colab Notebooks/NLU/transformers


In [7]:
!git checkout -f v2.7.0

Checking out files: 100% (474/474), done.
Note: checking out 'v2.7.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6f5a12a5 Release: v2.7.0


# Install packages required for running `run_glue.py`

In [8]:
!pip install -r ./examples/requirements.txt

     |████████████████████████████████| 204kB 3.5MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=b936d0e0b5f61938c69df5f1a9c59454a8638f30ec07537c311a7e1927be5d5d
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [9]:
!pip install boto3 filelock requests tqdm sentencepiece sacremoses tokenizers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 890kB 9.7MB/s 
     |████████████████████████████████| 3.8MB 24.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=aa4fad9e0efc8d0f74fb4b6752494f354b0e7451379c0ec94ebb320c00c168de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Download GLUE data

In [0]:
GLUE_DIR="/tmp/data"

In [11]:
!echo $GLUE_DIR

/tmp/data


In [12]:
!python ./utils/download_glue_data.py --help

usage: download_glue_data.py [-h] [--data_dir DATA_DIR] [--tasks TASKS]
                             [--path_to_mrpc PATH_TO_MRPC]

optional arguments:
  -h, --help            show this help message and exit
  --data_dir DATA_DIR   directory to save data to
  --tasks TASKS         tasks to download data for as a comma separated string
  --path_to_mrpc PATH_TO_MRPC
                        path to directory containing extracted MRPC data,
                        msr_paraphrase_train.txt and msr_paraphrase_text.txt


In [13]:
!python ./utils/download_glue_data.py --data_dir $GLUE_DIR --tasks MRPC

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!


# Compute RoBERTa score on MRPC task

In [0]:
TASK_NAME="MRPC"

In [15]:
!echo $TASK_NAME

MRPC


In [16]:
!echo $PYTHONPATH

/env/python


In [17]:
%set_env PYTHONPATH=$WORKING_DIR/transformers/src:/env/python

env: PYTHONPATH=/content/drive/My Drive/Colab Notebooks/NLU/transformers/src:/env/python


In [18]:
!echo $PYTHONPATH

/content/drive/My Drive/Colab Notebooks/NLU/transformers/src:/env/python


In [19]:
MODEL_DIR = os.path.join('/content', 'drive', 'My Drive', 'Colab Notebooks', 'NLU', 'models', 'baseline')
os.path.exists(MODEL_DIR)

True

In [20]:
!python ./examples/run_glue.py \
    --model_type roberta \
    --model_name_or_path /content/drive/My\ Drive/Colab\ Notebooks/NLU/models/baseline \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --output_dir glue_model \
    --overwrite_output_dir

2020-05-15 16:40:50.667376: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/15/2020 16:40:55 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/15/2020 16:40:55 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/Colab Notebooks/NLU/models/baseline/config.json
05/15/2020 16:40:55 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForTokenDiscrimination"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediat